In [2]:
import pentago
import pentago.agent
import pentago.controller

In [3]:
agents = [pentago.agent.RandomAgent()] * 2
view = pentago.controller.MemoryView()
controller = pentago.controller.Controller(agents, view)

for i in range(10):
    controller.play_game()

In [4]:
view.memory

deque([{'turn': 1, 'board': array([[0, 0, 0, 0, 0, 0],
               [0, 0, 0, 0, 0, 0],
               [0, 0, 0, 0, 0, 0],
               [0, 0, 0, 0, 0, 0],
               [0, 0, 0, 0, 0, 0],
               [0, 0, 0, 0, 0, 0]], dtype=int8), 'value': 0, 'action_values': None},
       {'turn': -1, 'board': array([[0, 0, 0, 0, 0, 1],
               [0, 0, 0, 0, 0, 0],
               [0, 0, 0, 0, 0, 0],
               [0, 0, 0, 0, 0, 0],
               [0, 0, 0, 0, 0, 0],
               [0, 0, 0, 0, 0, 0]], dtype=int8), 'value': 0, 'action_values': None},
       {'turn': 1, 'board': array([[ 0,  0,  0, -1,  0,  1],
               [ 0,  0,  0,  0,  0,  0],
               [ 0,  0,  0,  0,  0,  0],
               [ 0,  0,  0,  0,  0,  0],
               [ 0,  0,  0,  0,  0,  0],
               [ 0,  0,  0,  0,  0,  0]], dtype=int8), 'value': 0, 'action_values': None},
       {'turn': -1, 'board': array([[ 0,  0,  0, -1,  0,  1],
               [ 0,  0,  0,  0,  0,  0],
               [ 0, 